In [1]:
import pandas as pd
import numpy as np

import requests
from requests import get
from bs4 import BeautifulSoup

from time import sleep
from random import randint

## Scrapping process
using beautiful soap for the process.

In [2]:
# Creating the lists we want to write into
titles = []
years = []
time = []
imdb_ratings = []
metascores = []
votes = []
us_gross = []

In [3]:
# Getting English translated titles from the movies
headers = {'Accept-Language': 'en-US, en;q=0.5'}

In [4]:
pages = np.arange(1, 1001, 50)
pages

array([  1,  51, 101, 151, 201, 251, 301, 351, 401, 451, 501, 551, 601,
       651, 701, 751, 801, 851, 901, 951])

In [5]:
# Storing each of the urls of 50 movies 
for page in pages:
    # Getting the contents from the each url
    page = requests.get('https://www.imdb.com/search/title/?groups=top_1000&start=' + str(page) + '&ref_=adv_nxt', headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    # Aiming the part of the html we want to get the information from
    movie_div = soup.find_all('div', class_='lister-item mode-advanced')
    
    # Controling the loop’s rate by pausing the execution of the loop for a specified amount of time
    # Waiting time between requests for a number between 2-10 seconds
    sleep(randint(2,10))
    
    for container in movie_div:
        # Scraping the movie's name
        name = container.h3.a.text
        titles.append(name)
        
        # Scraping the movie's year
        year = container.h3.find('span', class_='lister-item-year').text
        years.append(year)
        
        # Scraping the movie's length
        runtime = container.find('span', class_='runtime').text if container.p.find('span', class_='runtime') else '-'
        time.append(runtime)
        
        # Scraping the rating
        imdb = float(container.strong.text)
        imdb_ratings.append(imdb)
        
        # Scraping the metascore
        m_score = container.find('span', class_='metascore').text if container.find('span', class_='metascore') else '-'
        metascores.append(m_score)
        
        # Scraping votes and gross earnings
        nv = container.find_all('span', attrs={'name':'nv'})
        vote = nv[0].text
        votes.append(vote)
        grosses = nv[1].text if len(nv) > 1 else '-'
        us_gross.append(grosses)

## Dataset implementation

In [11]:
movies = pd.DataFrame({'movie':titles,
                       'year':years,
                       'time_minute':time,
                       'imdb_rating':imdb_ratings,
                       'metascore':metascores,
                       'vote':votes,
                       'gross_earning':us_gross})

movies.head()

,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,Dune,(2021),155 min,8.4,75,"77,341",-
1,Shang-Chi and the Legend of the Ten Rings,(2021),132 min,7.9,71,"107,265",-
2,Casino Royale,(2006),144 min,8.0,80,"611,982",$167.45M
3,Skyfall,(2012),143 min,7.8,81,"657,440",$304.36M
4,Halloween,(1978),91 min,7.7,87,"247,231",$47.00M


In [12]:
movies.dtypes

movie             object
year              object
time_minute       object
imdb_rating      float64
metascore         object
vote              object
gross_earning     object
dtype: object

## Creating the Dataset

In [13]:
# Cleaning 'year' column
movies['year'] = movies['year'].str.extract('(\d+)').astype(int)
movies.head(3)

,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,Dune,2021,155 min,8.4,75,"77,341",-
1,Shang-Chi and the Legend of the Ten Rings,2021,132 min,7.9,71,"107,265",-
2,Casino Royale,2006,144 min,8.0,80,"611,982",$167.45M


In [14]:
# Cleaning 'time_minute' column
movies['time_minute'] = movies['time_minute'].str.extract('(\d+)').astype(int)
movies.head(3)

,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,Dune,2021,155,8.4,75,"77,341",-
1,Shang-Chi and the Legend of the Ten Rings,2021,132,7.9,71,"107,265",-
2,Casino Royale,2006,144,8.0,80,"611,982",$167.45M


In [15]:
# Cleaning 'metascore' column
movies['metascore'] = movies['metascore'].str.extract('(\d+)')
# convert it to float and if there are dashes turn it into NaN
movies['metascore'] = pd.to_numeric(movies['metascore'], errors='coerce')

In [16]:
# Cleaning 'vote' column
movies['vote'] = movies['vote'].str.replace(',', '').astype(int)
movies.head(3)

,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,Dune,2021,155,8.4,75.0,77341,-
1,Shang-Chi and the Legend of the Ten Rings,2021,132,7.9,71.0,107265,-
2,Casino Royale,2006,144,8.0,80.0,611982,$167.45M


In [17]:
# Cleaning 'gross_earning' column
# left strip $ and right strip M 
movies['gross_earning'] = movies['gross_earning'].map(lambda x: x.lstrip('$').rstrip('M'))
# convert it to float and if there are dashes turn it into NaN
movies['gross_earning'] = pd.to_numeric(movies['gross_earning'], errors='coerce')
movies.head(3)

,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,Dune,2021,155,8.4,75.0,77341,NaN
1,Shang-Chi and the Legend of the Ten Rings,2021,132,7.9,71.0,107265,NaN
2,Casino Royale,2006,144,8.0,80.0,611982,167.45


In [18]:
movies.dtypes

movie             object
year               int32
time_minute        int32
imdb_rating      float64
metascore        float64
vote               int32
gross_earning    float64
dtype: object

In [19]:
movies

,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,Dune,2021,155,8.4,75.0,77341,NaN
1,Shang-Chi and the Legend of the Ten Rings,2021,132,7.9,71.0,107265,NaN
2,Casino Royale,2006,144,8.0,80.0,611982,167.45
3,Skyfall,2012,143,7.8,81.0,657440,304.36
4,Halloween,1978,91,7.7,87.0,247231,47.00
...,...,...,...,...,...,...,...
995,Paan Singh Tomar,2012,135,8.2,NaN,34986,0.04
996,Udaan,2010,134,8.2,NaN,43971,0.01
997,The Breath,2009,128,8.0,NaN,32857,NaN
998,"Carry On, Munna Bhai",2006,144,8.1,NaN,45361,2.22


## Exporting to the CSV

In [20]:
movies.to_csv('movies.txt')

we can convert to the text if necessary